# Nofar Mahrabi 

In [27]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import re
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import KNNImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

### Functions in use

In [28]:
def plot_column_distribution(df, column_name):
    plt.figure(figsize=(10, 6))
    sns.histplot(df[column_name], bins=30, kde=True)
    plt.title(f'Distribution of {column_name}')
    plt.xlabel(column_name)
    plt.ylabel('Frequency')
    plt.show()

In [30]:
def fill_supply_score_knn(df, num_features, cat_features, n_neighbors=8):
    # Encode categorical features using One-Hot Encoding
    encoder = OneHotEncoder(drop='first', sparse=False)
    encoded_categorical = encoder.fit_transform(df[cat_features])
    encoded_categorical_df = pd.DataFrame(encoded_categorical, columns=encoder.get_feature_names_out(cat_features))
    
    # Combine numerical and encoded categorical features
    df_combined = pd.concat([df[num_features], encoded_categorical_df], axis=1)
    
    # Scale numerical features
    scaler = StandardScaler()
    df_scaled = scaler.fit_transform(df_combined)
    
    # Add the Supply_score column
    df_scaled = pd.DataFrame(df_scaled, columns=df_combined.columns)
    df_scaled['Supply_score'] = df['Supply_score']
    
    # Use KNNImputer to fill missing values
    imputer = KNNImputer(n_neighbors=n_neighbors)
    df_imputed = imputer.fit_transform(df_scaled)
    df_imputed = pd.DataFrame(df_imputed, columns=df_scaled.columns)
    
    # Update the original DataFrame with the filled Supply_score values
    df['Supply_score'] = df_imputed['Supply_score']
    
    return df

In [31]:
def clean_model_column(model, manufactor):
    if not isinstance(model, str):
        return None
    # Remove special characters
    model = re.sub(r'[^\w\s]', '', model)
    # Remove extra spaces
    model = re.sub(r'\s+', ' ', model).strip()
    # Remove manufacturer name
    manufactor = re.sub(r'[^\w\s]', '', manufactor)
    model = re.sub(fr'\b{manufactor}\b', '', model, flags=re.IGNORECASE).strip()
    # Remove years
    model = re.sub(r'\b\d{4}\b', '', model).strip()
    # If the cleaned string is empty, set to None or another appropriate value
    if model == '':
        model = None
    return model

In [32]:
def plot_histogram_with_kde(df, column_name):
    plt.figure(figsize=(10, 6))
    sns.histplot(data=df, x=column_name, bins=30, kde=True)
    plt.title(f'Distribution of {column_name.capitalize()}')
    plt.xlabel(column_name.capitalize())
    plt.ylabel('Frequency')
    plt.show()
    

### Get the data

In [33]:
file_path = 'dataset.csv'
df = pd.read_csv(file_path)

### Data Preparation Function
The `prepare_data` function cleans and preprocesses the dataset to ensure it's ready for analysis and modeling.
The key operations performed by the function include:
1. **Data Cleaning**
2. **Removes duplicate**
3. **Handling Missing Values**
4. **Type conversion variables**
5. **Outlier Removal**


In [93]:
def prepare_data(df):
    
    #Create a copy of the original data
    df = df.copy()
    #Remove full duplicates
    df = df.drop_duplicates(keep='first')
    
    # Removing the test column with more than 90% missing values ​​without affecting the price column.
    # Removing the area column that also has missing values ​​while the city column has more influence on the price.
    columns_to_drop = ['Test', 'Area']
    # Drop the specified columns
    df.drop(columns=columns_to_drop, inplace=True)
    
    # Convert numeric values
    df['Km'] = pd.to_numeric(df['Km'], errors='coerce')
    df['capacity_Engine'] = pd.to_numeric(df['capacity_Engine'], errors='coerce')
    # Merge identical categories
    df['Engine_type'] = df['Engine_type'].replace(['היבריד'], 'היברידי')
    # Combined the rare categories into 'Other' to simplify the data and reduce noise from infrequent categories."
    df['Engine_type'] = df['Engine_type'].replace(['גז', 'טורבו דיזל', 'חשמלי'], 'אחר')

    # Remove commas from columns
    df['Km'] = df['Km'].replace(',', '', regex=True)
    df['capacity_Engine'] = df['capacity_Engine'].replace(',', '', regex=True)
    
    # Handle missing values ​​using groupby and fill in the median, average or common value accordingly
    df['capacity_Engine'] = df.groupby('manufactor')['capacity_Engine'].transform(lambda x: x.fillna(x.median()))  
    df['Engine_type'] = df.groupby(['model', 'Year'], group_keys=False)['Engine_type'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'אחר'))
    df['Km'] = df.groupby('Year')['Km'].transform(lambda x: x.fillna(x.mean())).astype(int)
    df = df.dropna(subset=['capacity_Engine'])
    
    # Ensure capacity_Engine is of integer type
    df['capacity_Engine'] = df['capacity_Engine'].astype(int)
    
    # Handle missing values in categorical columns
    mode_color = df['Color'].mode()[0]
    df['Color'].fillna(mode_color, inplace=True)
    df['Gear'] = df['Gear'].fillna(df['Gear'].mode()[0])
    # Fill missing values in 'Prev_ownership' based on 'Curr_ownership'
    df['Prev_ownership'].fillna(df['Curr_ownership'], inplace=True)
    # Fill missing values in 'Curr_ownership' based on 'Prev_ownership'
    df['Curr_ownership'].fillna(df['Prev_ownership'], inplace=True)
    # For remaining missing values, fill with the most common value 'פרטית'
    df['Prev_ownership'].fillna('פרטית', inplace=True)
    df['Curr_ownership'].fillna('פרטית', inplace=True)
    median_pic_num = df['Pic_num'].median() 
    df['Pic_num'].fillna(median_pic_num, inplace=True)
    

    # Run the function on the 'model' column to handle the values ​​and try to reach a uniform format
    df['model'] = df.apply(lambda row: clean_model_column(row['model'], row['manufactor']), axis=1)
    df['model'] = df['model'].replace('none', np.nan)
    df = df.dropna(subset=['model'])

    # Handle missing values ​​in the Supply_score column by the KNN algorithm
    num_features = ['Year', 'Hand', 'capacity_Engine', 'Price', 'Km', 'Pic_num']
    cat_features = ['Gear', 'Engine_type']
    df = fill_supply_score_knn(df, num_features, cat_features)


    # Convert categorical columns to categories
    df['manufactor'] = df['manufactor'].astype('category')
    df['model'] = df['model'].astype('category')
    df['Gear'] = df['Gear'].astype('category')
    df['Engine_type'] = df['Engine_type'].astype('category')
    df['Prev_ownership'] = df['Prev_ownership'].astype('category')
    df['Curr_ownership'] = df['Curr_ownership'].astype('category')
    df['City'] = df['City'].astype('category')
    df['Color'] = df['Color'].astype('category')

    # Removes outliers from the dataset by applying filters based on visual analysis (KDE histograms) and personal judgment. 
    # The goal is to retain only relevant and reasonable data points, ensuring higher data quality for further analysis and modeling.
   
    df = df[df['Year'] > 2000]
    df = df[df['Hand'] <= 6]
    df = df[df['Pic_num'] < 10]
    df = df[(df['Km'] > 0) & (df['Km'] < 273000)]
    df = df[(df['capacity_Engine'] > 150) & (df['capacity_Engine'] < 8000)]
    df = df[df['Supply_score'] < 2000]
    
    
    return df




In [94]:
# Prepare the data
df_processed = prepare_data(df)

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


## **Data Preprocessing**


Checking if there are still missing values ​​after running the function

In [95]:
df_processed.isnull().sum().sort_values()

manufactor         0
Color              0
Description        0
Repub_date         0
Cre_date           0
Pic_num            0
Price              0
City               0
Curr_ownership     0
Prev_ownership     0
Engine_type        0
capacity_Engine    0
Gear               0
Hand               0
model              0
Year               0
Km                 0
Supply_score       0
dtype: int64

In [96]:
df_processed = df_processed.dropna(subset=['Supply_score'])


In [97]:
df_processed.describe()

,Year,Hand,capacity_Engine,Price,Pic_num,Km,Supply_score
count,1172.000000,1172.000000,1172.000000,1172.000000,1172.000000,1172.000000,1172.000000
mean,2014.650171,2.233788,1678.891638,51059.929181,2.129693,113647.168089,366.508532
std,3.341197,1.097292,716.983472,22292.222171,2.113009,59780.869260,340.123036
min,2002.000000,1.000000,800.000000,18200.000000,0.000000,49.000000,0.000000
25%,2013.000000,1.000000,1200.000000,30475.000000,1.000000,72150.000000,136.093750
50%,2015.000000,2.000000,1500.000000,48000.000000,1.000000,119807.000000,228.125000
75%,2017.000000,3.000000,1800.000000,68000.000000,3.000000,150460.000000,560.156250
max,2023.000000,6.000000,5500.000000,99960.000000,9.000000,270000.000000,1980.000000


In [99]:
df_processed.isnull().sum().sort_values()

manufactor         0
Color              0
Description        0
Repub_date         0
Cre_date           0
Pic_num            0
Price              0
City               0
Curr_ownership     0
Prev_ownership     0
Engine_type        0
capacity_Engine    0
Gear               0
Hand               0
model              0
Year               0
Km                 0
Supply_score       0
dtype: int64

# **--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

###  I will split the data at this stage before performing normalization to prevent data leakage or bias.

In [102]:
# Split the data into features and target
X = df_processed.drop(columns=['Price'])
y = df_processed['Price']
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [103]:
# Define numeric and categorical columns   
categorical_features = ['Gear', 'Engine_type', 'manufactor', 'model', 'Prev_ownership', 'Curr_ownership', 'Color','City']
numeric_features = ['Year', 'Hand', 'capacity_Engine', 'Km', 'Pic_num', 'Supply_score']

### Data Preprocessing Steps:
This section describes the data preprocessing steps using scikit-learn's `Pipeline` and `ColumnTransformer`:

1. **Numerical Features:**
   - Impute missing values with the mean.
   - Standardize features (mean = 0, std = 1).

2. **Categorical Features:**
   - Impute missing values with 'most_frequent'.
   - One-hot encode categories, ignoring unknowns.

3. **Combining Transformations:**
   - Use `ColumnTransformer` to apply different transformations to numerical and categorical features.

In [104]:
from sklearn.preprocessing import PolynomialFeatures

# Define the preprocessing for numerical features with Polynomial Features
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures(degree=2, include_bias=False))])  # Adding polynomial features

# Define the preprocessing for categorical features
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent', fill_value='missing')),  # Handle missing values
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Combine the transformers into a ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])


 The choice to use polynomial regression stems from its ability to capture non-linear relationships between the independent variables and the dependent variable, which can improve model performance. Using polynomial features also allows for capturing interactions between different features, providing a richer representation of the data patterns.

### Model and Hyperparameter Tuning

In this section, I define and train the ElasticNet regression model. 
To optimize the model's performance, we use RandomizedSearchCV for hyperparameter tuning, searching over a range of values for the alpha and l1_ratio parameters.

In [105]:
# Define the Elastic Net model
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', ElasticNet(max_iter=10000, tol=1e-4, random_state=42))])

# Define the hyperparameter search space
param_distributions = {
    'regressor__alpha': uniform(0.01, 20),  # Extended range for Regularization strength
    'regressor__l1_ratio': uniform(0, 1)    # Mix between L1 and L2 regularization
}

# Setup the RandomizedSearchCV
random_search = RandomizedSearchCV(model, param_distributions, n_iter=500, cv=5, scoring='neg_mean_squared_error', random_state=42)

# Fit the RandomizedSearchCV to the data
random_search.fit(X_train, y_train)

# Print the best parameters found
print("Best parameters found: ", random_search.best_params_)

# Predict on the training data (or you can use validation data if available)
y_train_pred = random_search.predict(X_train)

# Calculate the RMSE
rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
print(f'Root Mean Squared Error (RMSE) on training data: {rmse:.2f}')

Best parameters found:  {'regressor__alpha': 1.7169933615728772, 'regressor__l1_ratio': 0.9968742518459474}
Root Mean Squared Error (RMSE) on training data: 9102.97


### K-Fold Cross-Validation

In [106]:
from sklearn.model_selection import cross_val_score

# Perform 10-fold cross-validation
cv_scores = cross_val_score(random_search.best_estimator_, X_train, y_train, cv=10, scoring='neg_mean_squared_error')

# Calculate RMSE for each fold and the mean RMSE
rmse_scores = np.sqrt(-cv_scores)
mean_rmse = np.mean(rmse_scores)
std_rmse = np.std(rmse_scores)

print(f'10-Fold Cross-Validation RMSE: {rmse_scores}')
print(f'Mean RMSE: {mean_rmse:.2f}')
print(f'Standard Deviation of RMSE: {std_rmse:.2f}')

10-Fold Cross-Validation RMSE: [10869.63771705 10948.6540873  13964.81813114 12108.56648101
 11074.88008659 12009.44414223 11488.36700983 12430.24286249
 13675.61142738 10300.4723855 ]
Mean RMSE: 11887.07
Standard Deviation of RMSE: 1145.72


### Model Performance Metrics
#### In this section, I evaluate the performance of our trained ElasticNet model using several relevant performance metrics. These metrics provide a comprehensive view of how well the model is performing on the test data

In [107]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Predict on the test data
y_test_pred = random_search.predict(X_test)

# Calculate RMSE
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
print(f'Root Mean Squared Error (RMSE) on test data: {test_rmse:.2f}')

# Calculate MAE
test_mae = mean_absolute_error(y_test, y_test_pred)
print(f'Mean Absolute Error (MAE) on test data: {test_mae:.2f}')

# Calculate Adjusted R² score
n = len(y_test)
p = X_test.shape[1]
test_adj_r2 = 1 - (1 - test_r2) * (n - 1) / (n - p - 1)
print(f'Adjusted R² Score on test data: {test_adj_r2:.2f}')

Root Mean Squared Error (RMSE) on test data: 11067.56
Mean Absolute Error (MAE) on test data: 8702.04
Adjusted R² Score on test data: 0.72


### Identifying the Top 5 Most Influential Features

In [113]:
from tabulate import tabulate


# Extract the model with the best parameters
best_model = random_search.best_estimator_.named_steps['regressor']

# Get the preprocessor step
preprocessor = random_search.best_estimator_.named_steps['preprocessor']

# Get numerical feature names after polynomial features
numeric_feature_names = preprocessor.transformers_[0][1].named_steps['poly'].get_feature_names_out(numeric_features)

# Get categorical feature names
categorical_feature_names = preprocessor.transformers_[1][1].get_feature_names_out(categorical_features)

# Combine all feature names
feature_names = np.concatenate([numeric_feature_names, categorical_feature_names])

# Get coefficients
coefficients = best_model.coef_

# Create a DataFrame to display feature importances
feature_importance = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': coefficients
})

# Sort by absolute value of coefficients
feature_importance['Absolute Coefficient'] = feature_importance['Coefficient'].abs()
feature_importance = feature_importance.sort_values(by='Absolute Coefficient', ascending=False)

# Add a column for Positive/Negative impact
feature_importance['Impact'] = feature_importance['Coefficient'].apply(lambda x: 'Positive' if x > 0 else 'Negative')

# Get top 5 features
top_5_features = feature_importance.head(5).set_index('Feature')

# Align values to the left
pd.set_option('display.colheader_justify', 'left')  # for left align
pd.set_option('display.float_format', '{:.2f}'.format)  # format floating numbers

# Print the top 5 features in a clear and readable format using tabulate
print("Top 5 Features with the Largest Impact:\n")
print(tabulate(top_5_features, headers='keys', tablefmt='pretty', numalign='left', stralign='left'))

Top 5 Features with the Largest Impact:

+--------------------+---------------------+----------------------+----------+
| Feature            | Coefficient         | Absolute Coefficient | Impact   |
+--------------------+---------------------+----------------------+----------+
| Year               | 16658.294372806315  | 16658.294372806315   | Positive |
| model_IMIEV        | 13340.798668160423  | 13340.798668160423   | Positive |
| manufactor_מיני    | 12214.194902776027  | 12214.194902776027   | Positive |
| model_סונטה        | 11748.931323701776  | 11748.931323701776   | Positive |
| model_לנסר ספורטבק | -11081.975579066653 | 11081.975579066653   | Negative |
+--------------------+---------------------+----------------------+----------+
